In [6]:
"""
Basic Example
""" 

from scapy.all import *

# Means Raw captualed by TCP and then captulated by IP
p = IP(dst = '192.168.1.2') / TCP(dport = 80) / Raw(b'Some raw bytes')
# to see packet content as bytes use bytes(p) not str(p)
#sr1(p)
p.show()
print("\nHexdump of frame:")
hexdump(p)

print("\nBytes of frame:")
bytes(p)

###[ IP ]###
  version   = 4
  ihl       = None
  tos       = 0x0
  len       = None
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = 6
  chksum    = None
  src       = 172.17.0.2
  dst       = 192.168.1.2
  \options   \
###[ TCP ]###
     sport     = 20
     dport     = 80
     seq       = 0
     ack       = 0
     dataofs   = None
     reserved  = 0
     flags     = S
     window    = 8192
     chksum    = None
     urgptr    = 0
     options   = {}
###[ Raw ]###
        load      = 'Some raw bytes'

Hexdump of frame:
0000   45 00 00 36 00 01 00 00  40 06 0D 04 AC 11 00 02   E..6....@.......
0010   C0 A8 01 02 00 14 00 50  00 00 00 00 00 00 00 00   .......P........
0020   50 02 20 00 DF AA 00 00  53 6F 6D 65 20 72 61 77   P. .....Some raw
0030   20 62 79 74 65 73                                   bytes

Bytes of frame:


b'E\x00\x006\x00\x01\x00\x00@\x06\r\x04\xac\x11\x00\x02\xc0\xa8\x01\x02\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00\xdf\xaa\x00\x00Some raw bytes'

In [5]:
"""
Basic Example to generate 802.11 Beacon frame
"""

from scapy.all import Dot11,Dot11Beacon,Dot11Elt,RadioTap,sendp,hexdump

netSSID = 'testSSID'       #Network name here

dot11 = Dot11(type=0, subtype=8, addr1='ff:ff:ff:ff:ff:ff',
addr2='22:22:22:22:22:22', addr3='33:33:33:33:33:33')
beacon = Dot11Beacon(cap='ESS+privacy')
essid = Dot11Elt(ID='SSID',info=netSSID, len=len(netSSID))
rsn = Dot11Elt(ID='RSNinfo', info=(
'\x01\x00'                 #RSN Version 1
'\x00\x0f\xac\x02'         #Group Cipher Suite : 00-0f-ac TKIP
'\x02\x00'                 #2 Pairwise Cipher Suites (next two lines)
'\x00\x0f\xac\x04'         #AES Cipher
'\x00\x0f\xac\x02'         #TKIP Cipher
'\x01\x00'                 #1 Authentication Key Managment Suite (line below)
'\x00\x0f\xac\x02'         #Pre-Shared Key
'\x00\x00'))               #RSN Capabilities (no extra capabilities)

#frame = RadioTap()/dot11/beacon/essid/rsn
# frame encapsulation
frame = dot11/beacon/essid/rsn

frame.show()
print("\nHexdump of frame:")
hexdump(frame)

###[ 802.11 ]###
  subtype   = 8
  type      = Management
  proto     = 0
  FCfield   = 
  ID        = 0
  addr1     = ff:ff:ff:ff:ff:ff
  addr2     = 22:22:22:22:22:22
  addr3     = 33:33:33:33:33:33
  SC        = 0
  addr4     = 00:00:00:00:00:00
  FCS       = None
###[ 802.11 Beacon ]###
     timestamp = 0
     beacon_interval= 100
     cap       = ESS+privacy
###[ 802.11 Information Element ]###
        ID        = SSID
        len       = 8
        info      = 'testSSID'
###[ 802.11 Information Element ]###
           ID        = RSNinfo
           len       = None
           info      = b'\x01\x00\x00\x0f\xac\x02\x02\x00\x00\x0f\xac\x04\x00\x0f\xac\x02\x01\x00\x00\x0f\xac\x02\x00\x00'

Hexdump of frame:
0000   80 00 00 00 FF FF FF FF  FF FF 22 22 22 22 22 22   ..........""""""
0010   33 33 33 33 33 33 00 00  00 00 00 00 00 00 00 00   333333..........
0020   64 00 11 00 00 08 74 65  73 74 53 53 49 44 30 18   d.....testSSID0.
0030   01 00 00 0F AC 02 02 00  00 0F AC 04 00 0F AC 02 

In [4]:
"""
Basic Example to generate 802.11 data frame header with addr4
"""
from scapy.all import Dot11,Dot11Beacon,Dot11Elt,RadioTap,sendp,hexdump

# To show addr4 => type=2, from-DS, to-DS all to be set
dot11 = Dot11(type=2, subtype=0, FCfield='from-DS+to-DS', addr1='ff:ff:ff:ff:ff:ff',
addr2='22:22:22:22:22:22', addr3='33:33:33:33:33:33', addr4='55:55:55:55:55:55')

frame = dot11

frame.show()
print("\nHexdump of frame:")
hexdump(frame)

###[ 802.11 ]###
  subtype   = 0
  type      = Data
  proto     = 0
  FCfield   = to-DS+from-DS
  ID        = 0
  addr1     = ff:ff:ff:ff:ff:ff
  addr2     = 22:22:22:22:22:22
  addr3     = 33:33:33:33:33:33
  SC        = 0
  addr4     = 55:55:55:55:55:55
  FCS       = None

Hexdump of frame:
0000   08 03 00 00 FF FF FF FF  FF FF 22 22 22 22 22 22   ..........""""""
0010   33 33 33 33 33 33 00 00  55 55 55 55 55 55         333333..UUUUUU


In [8]:
"""
Creating a customized packet
"""
from scapy.all import *
 
class Dot11EltRates(Packet):
    """ Our own definition for the supported rates field """
    name = "802.11 Rates Information Element"
    # Our Test STA supports the rates 6, 9, 12, 18, 24, 36, 48 and 54 Mbps
    supported_rates = [0x0c, 0x12, 0x18, 0x24, 0x30, 0x48, 0x60, 0x6c]
    fields_desc = [ByteField("ID", 1), ByteField("len", len(supported_rates))]
    for index, rate in enumerate(supported_rates):
        fields_desc.append(ByteField("supported_rate{0}".format(index + 1),
                                     rate))
 
packet = Dot11(
    addr1="00:a0:57:98:76:54",
    addr2="00:a0:57:12:34:56",
    addr3="00:a0:57:98:76:54") / Dot11AssoReq(
        cap=0x1100, listen_interval=0x00a) / Dot11Elt(
            ID=0, info="MY_BSSID")
packet /= Dot11EltRates()
#sendp(packet, iface="wlp0s29u1u7")
packet.show()

###[ 802.11 ]###
  subtype   = 0
  type      = Management
  proto     = 0
  FCfield   = 
  ID        = 0
  addr1     = 00:a0:57:98:76:54
  addr2     = 00:a0:57:12:34:56
  addr3     = 00:a0:57:98:76:54
  SC        = 0
  addr4     = 00:00:00:00:00:00
  FCS       = None
###[ 802.11 Association Request ]###
     cap       = ESS+privacy
     listen_interval= 10
###[ 802.11 Information Element ]###
        ID        = SSID
        len       = None
        info      = 'MY_BSSID'
###[ 802.11 Rates Information Element ]###
           ID        = 1
           len       = 8
           supported_rate1= 12
           supported_rate2= 18
           supported_rate3= 24
           supported_rate4= 36
           supported_rate5= 48
           supported_rate6= 72
           supported_rate7= 96
           supported_rate8= 108


In [3]:
"""
Test myMBSSID packet
Mapping to Packet #1 of Network_Join_Nokia_Mobile.pcap
"""
from scapy.all import *

dot11 = Dot11(type='Management', subtype=8, addr1='ff:ff:ff:ff:ff:ff',
addr2='22:22:22:22:22:22', addr3='33:33:33:33:33:33', SC=0xf010)
# Fixed IE in beacon (timestamp, beacon_interval, cap)
beacon = Dot11Beacon(timestamp=0x00000002691a2184, cap='ESS+privacy+short-slot')
# TLV IE in beacon
netSSID="martinet3"
essid = Dot11Elt(ID='SSID',info=netSSID, len=len(netSSID))
supRates=Dot11Elt(ID='Rates', info=(
'\x82\x84\x8b\x96\x24\x30\x48\x6c'
))
dsSetIe=Dot11Elt(ID='DSset', info=(
'\x0b'
))
timIe=Dot11Elt(ID='TIM', info=(
'\x00\x01\x00\x00'
))
erpIe1=Dot11Elt(ID=0x2a, info=(
'\x04'
))
erpIe2=Dot11Elt(ID=0x2f, info=(
'\x04'
))
esRateIe=Dot11Elt(ID='ESRates', info=(
'\x0c\x12\x18\x60'
))

frame = dot11/beacon/essid/supRates/dsSetIe/timIe/erpIe1/erpIe2/esRateIe
frame.show()
hexdump(frame)

# Write frame to PCAP file
pktdump = PcapWriter("./result/banana.pcap", append=False, sync=True)
pktdump.write(frame)
beacon = Dot11Beacon(timestamp=(0x00000002691a2184+100), cap='ESS+privacy+short-slot')
frame = dot11/beacon/essid/supRates/dsSetIe/timIe/erpIe1/erpIe2/esRateIe
pktdump.write(frame)

print('done')

###[ 802.11 ]###
  subtype   = 8
  type      = Management
  proto     = 0
  FCfield   = 
  ID        = 0
  addr1     = ff:ff:ff:ff:ff:ff
  addr2     = 22:22:22:22:22:22
  addr3     = 33:33:33:33:33:33
  SC        = 61456
  addr4     = 00:00:00:00:00:00
  FCS       = None
###[ 802.11 Beacon ]###
     timestamp = 10353254788
     beacon_interval= 100
     cap       = short-slot+ESS+privacy
###[ 802.11 Information Element ]###
        ID        = SSID
        len       = 9
        info      = 'martinet3'
###[ 802.11 Information Element ]###
           ID        = Rates
           len       = None
           info      = b'\x82\x84\x8b\x96$0Hl'
###[ 802.11 Information Element ]###
              ID        = DSset
              len       = None
              info      = '\x0b'
###[ 802.11 Information Element ]###
                 ID        = TIM
                 len       = None
                 info      = '\x00\x01\x00\x00'
###[ 802.11 Information Element ]###
                    ID        